# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import time
import collections


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import the csv file that was created in part 1 of assignment
cities_weather = pd.read_csv("exported_data/cities_part_1.csv", encoding='utf-8')
cities_weather


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rikitea,1,PF,1647844974,73,-23.1203,-134.9692,79.84,14.16
1,Mpika,100,ZM,1647844975,94,-11.8343,31.4529,61.97,7.65
2,Lebu,47,CL,1647844976,92,-37.6167,-73.6500,55.96,4.36
3,Bluff,10,NZ,1647844976,85,-46.6000,168.3333,60.48,14.74
4,San Luis,0,AR,1647844977,55,-33.2950,-66.3356,55.62,9.46
...,...,...,...,...,...,...,...,...,...
543,Mehamn,100,NO,1647845353,56,71.0357,27.8492,42.71,34.31
544,Iralaya,100,HN,1647845354,81,15.0000,-83.2333,77.59,13.09
545,Conde,100,BR,1647845354,94,-7.2597,-34.9075,76.46,3.44
546,Yuncheng,100,CN,1647845355,54,35.0231,110.9928,55.20,2.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
#Storing the latitude and longitude in the locations
locations = cities_weather[['Lat', 'Lng']]

#Fill missing (NaN) values and coversion to a float.
humidity = cities_weather["Humidity"].astype(float)

In [5]:
#Plot the heatmap
fig = gmaps.figure()
max_humidity = np.max(humidity)

#Create the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                  max_intensity=100, dissipating=False, 
                                   point_radius=1, opacity=0.5)

#Add layer

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# #Maximum temperature lower than 80 degress, however higher than 70.
# #Wind speed less than 10 mph.
# #No cloudiness
# #Dropping any row that don't contain any of these conditions.

ideal_cities = cities_weather.copy()
ideal_cities = ideal_cities[ideal_cities['Max Temp'] >= 70]
ideal_cities = ideal_cities[ideal_cities['Max Temp'] <= 80]
ideal_cities = ideal_cities[ideal_cities['Wind Speed'] <= 10]
ideal_cities = ideal_cities[ideal_cities['Cloudiness'] == 0]
ideal_cities = ideal_cities.dropna()
ideal_cities = ideal_cities.reset_index()
ideal_cities.head(10)

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,9,Busselton,0,AU,1647844981,38,-33.6500,115.3333,77.94,8.05
1,68,Cape Town,0,ZA,1647844743,82,-33.9258,18.4232,70.12,9.22
2,90,Pochutla,0,MX,1647845033,65,15.7432,-96.4661,76.82,1.45
3,245,Umluj,0,SA,1647845140,49,25.0213,37.2685,72.07,4.18
4,289,Mīrābād,0,AF,1647845170,11,30.4362,61.8383,75.54,1.90
5,342,Acapulco de Juárez,0,MX,1647844673,78,16.8634,-99.8901,75.02,0.00
6,394,Champerico,0,GT,1647845244,70,14.3000,-91.9167,79.39,5.03
7,401,Caravelas,0,BR,1647845249,86,-17.7125,-39.2481,74.62,1.79
8,446,Ormara,0,PK,1647845070,68,25.2088,64.6357,76.89,8.16
9,507,Makueni Boma,0,KE,1647845326,44,-1.8041,37.6203,77.68,5.23


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Creating the hotel_df and add the hotel name column
hotel_df = ideal_cities
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,9,Busselton,0,AU,1647844981,38,-33.6500,115.3333,77.94,8.05,
1,68,Cape Town,0,ZA,1647844743,82,-33.9258,18.4232,70.12,9.22,
2,90,Pochutla,0,MX,1647845033,65,15.7432,-96.4661,76.82,1.45,
3,245,Umluj,0,SA,1647845140,49,25.0213,37.2685,72.07,4.18,
4,289,Mīrābād,0,AF,1647845170,11,30.4362,61.8383,75.54,1.90,
5,342,Acapulco de Juárez,0,MX,1647844673,78,16.8634,-99.8901,75.02,0.00,
6,394,Champerico,0,GT,1647845244,70,14.3000,-91.9167,79.39,5.03,
7,401,Caravelas,0,BR,1647845249,86,-17.7125,-39.2481,74.62,1.79,
8,446,Ormara,0,PK,1647845070,68,25.2088,64.6357,76.89,8.16,
9,507,Makueni Boma,0,KE,1647845326,44,-1.8041,37.6203,77.68,5.23,


In [8]:
#Setting the base_url
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

#Setting the parameters, 5000 metres
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key
}

count = 0

# Using the iterrows
for index, row in hotel_df.iterrows():
    
    #Use city to get name from list
    city = row['City']
    
    #Adding the cities to lat and lng params
    params["location"] = f"{row['Lat']}, {row['Lng']}"
    
    #start the count
    count += 1
    
    #print data
    print(f"acquiring data for {city}")
    
    #making the request
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        #Go through list, add the first hotel result to hotel_df
        hotel_df.loc[index,'Hotel Name'] = hotel_data['results'][0]['name']
        print('--------')
        print(f'Hotel found')
        print('--------')
        
    except:
        print('--------')
        hotel_df.loc[index, 'Hotel Name'] = "no hotel in range"
        print("Oops, hotel not found")
        print('--------')
        
    #To not have time out errors    
    time.sleep(1)
    
print(f'searching complete')
   

acquiring data for Busselton
--------
Hotel found
--------
acquiring data for Cape Town
--------
Hotel found
--------
acquiring data for Pochutla
--------
Oops, hotel not found
--------
acquiring data for Umluj
--------
Hotel found
--------
acquiring data for Mīrābād
--------
Oops, hotel not found
--------
acquiring data for Acapulco de Juárez
--------
Hotel found
--------
acquiring data for Champerico
--------
Hotel found
--------
acquiring data for Caravelas
--------
Hotel found
--------
acquiring data for Ormara
--------
Hotel found
--------
acquiring data for Makueni Boma
--------
Hotel found
--------
searching complete


In [9]:
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,9,Busselton,0,AU,1647844981,38,-33.6500,115.3333,77.94,8.05,Bayview Geographe Resort
1,68,Cape Town,0,ZA,1647844743,82,-33.9258,18.4232,70.12,9.22,Grande Kloof Boutique Hotel
2,90,Pochutla,0,MX,1647845033,65,15.7432,-96.4661,76.82,1.45,no hotel in range
3,245,Umluj,0,SA,1647845140,49,25.0213,37.2685,72.07,4.18,Juman Hotel
4,289,Mīrābād,0,AF,1647845170,11,30.4362,61.8383,75.54,1.90,no hotel in range
5,342,Acapulco de Juárez,0,MX,1647844673,78,16.8634,-99.8901,75.02,0.00,Hotel Krystal Beach Acapulco
6,394,Champerico,0,GT,1647845244,70,14.3000,-91.9167,79.39,5.03,Hotel Miramar
7,401,Caravelas,0,BR,1647845249,86,-17.7125,-39.2481,74.62,1.79,Pousada Liberdade
8,446,Ormara,0,PK,1647845070,68,25.2088,64.6357,76.89,8.16,Alqadri hotel zero point ormara
9,507,Makueni Boma,0,KE,1647845326,44,-1.8041,37.6203,77.68,5.23,FORT HOTEL mami


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)
    

# Display figure
fig